In [11]:
import openai
from dotenv import load_dotenv
import os
from langchain.vectorstores import Chroma
import gradio as gr
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [12]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [13]:
vector_store = Chroma(
    embedding_function=embedding_function,
    persist_directory="../chroma",
    collection_name="osagyefo_v1",
)

In [14]:
retriever = vector_store.as_retriever(
    search_type="mmr",  # Maximal Marginal Relevance
    search_kwargs={"k": 5, "lambda_mult": 0.5}
)

In [15]:
print(vector_store._collection.count())  

1283


In [16]:
retriever.get_relevant_documents("my mother did not give me food")

[Document(metadata={'chapter_title': 'CHAPTER EIGHT', 'doc_type': 'act', 'total_pages': 115, 'file_name': 'criminal offences act.pdf'}, page_content='CHAPTER EIGHT\nPublic Nuisance\nHindering Burials\n285.   Hindering burial of dead body\nA person who unlawfully hinders the burial of the d ead body of a person, or without lawful authority\ndisinters, dissects or harms the dead body of a person, or being under a duty to cause the dead body of a\nperson to be buried, fails to perform that duty commits a misdemeanour.\nUnwholesome Food\n286.   Selling unwholesome food\nA person who sells, or prepares or offers for sale, as being fit for consumption as food or drink, a thing\nwhich that person knows or has reason to believe that it is in a condition of putrefaction, adulteration, or\nother cause, as to be likely to be noxious to health, commits a misdemeanour.\nNoxious Trade\n287.   Carrying on of noxious trade, interference with public rights\nA person who, without lawful authority or exc

In [17]:
load_dotenv(override=True)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

model_1 = "gemma2-9b-it"
client1 = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY"),
)

model_2 = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
client2 = openai.OpenAI(
  api_key=os.environ.get("TOGETHER_API_KEY"),
  base_url="https://api.together.xyz/v1",
)

PIPELINE

In [18]:
"""
LLM 1 will rephrase every question into 3 different similar questions--gemma2-9b-it

"""

def rephrase_query(user_query):
    pdf_path = "../docs"
    prompt_path = "../prompts/rephaser_llm_template.txt"
    
    with open(prompt_path, "r", encoding="utf-8") as f:
        system_prompt = f.read().strip()
        system_prompt += "Your knowledge is limited strictly to the content of the following files:"
        for filename in os.listdir(pdf_path):
            system_prompt += filename + "\n"


    message = [{'role': "system", "content": system_prompt}, {'role': "system", "content": user_query}]
    rephrased_query = client1.chat.completions.create(model=model_1, messages=message)
    rephrased_query = rephrased_query.choices[0].message.content

    return rephrased_query

In [19]:
# test
rephrased_query = rephrase_query("i mistakenly hit someone with my car")
print(rephrased_query)

context = "" 
relevant_docs = retriever.get_relevant_documents(rephrased_query)

for doc in relevant_docs:
    context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

['Negligence resulting in bodily harm by motor vehicle', 'Consequences of an unintentional vehicular collision', 'Liability under Ghanaian law for accidental striking by motor vehicle'] 



In [20]:
context

'[Source: criminal offences act.pdf, Chapter: CHAPTER EIGHT]\n(s)        in a town wilfully or negligently causes or pe rmits a vehicle, or a thing carried by the\nvehicle, or cattle, to damage a drain, ditch, or trench, at the side of a street, or a bridge, or a\npart of it respectively; or\n            (t)        in a town, without the written consent of the local authority or the Ghana Highway\nAuthority, wilfully displaces or takes up or damages the pavement, stones, or material of a\npublic way, or attempts to change or obstruct an outer course; or91(91)\n            (u)      assembles with other persons in a public place, or in an open space near a public place, for an\nidle, a vicious, or a disorderly purpose, or ot herwise than in the regular performance, or in\npursuance, of a lawful calling or object, to the annoyance or  obstruction of a passenger or\nperson frequenting the public place, or of a person living in the neighbourhood of the public\nplace or open space and does n

In [31]:
def chat_osagyefo(user_query, history):
    
    prompt_path = "../prompts/main_prompt_template.txt"

    rephrased_query = rephrase_query(user_query)
    relevant_docs = retriever.get_relevant_documents(rephrased_query)

    print("RELEVANT DOCS: ", relevant_docs)
    context = ''
    for doc in relevant_docs:
        context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

    # loading the prompt
    with open(prompt_path, "r", encoding="utf-8") as f:
        osagyefo_system_prompt = f.read().strip()
    osagyefo_system_prompt += f"Use the following context to answer:\n{context}"


    history = [
        {'role': h['role'], 'content': h['content'] }
        for h in history
        if 'role' in h and 'content' in h
    ]

    msg_to_model = [{"role": "system", "content": osagyefo_system_prompt}] + history + [{"role": "user", "content": user_query}]

    model = client2.chat.completions.create(
        model=model_2,
        messages=msg_to_model,
    )

    osagyefo_response = model.choices[0].message.content
    

    history.append({"role": "user", "content": user_query})
    history.append({"role": "assistant", "content": osagyefo_response})

    print('history', history)
    
    return osagyefo_response

In [32]:
demo_chatbot = gr.ChatInterface(chat_osagyefo,
                                title="OSAGYEFO ESQ.",
                                description="Welcome! Put forth your legal challenges. ")

c:\Users\DeLL\Desktop\anything_py\osagyefo_ai\.venv\lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [ ]:
demo_chatbot.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


RELEVANT DOCS:  [Document(metadata={'total_pages': 161, 'chapter_title': 'Chapter 25', 'doc_type': 'constitution', 'file_name': 'ghana_constitution.pdf'}, page_content='designate . \nTHE OFFICIAL OATH  \nI, ___________________ do (in the name of the Almighty God swear) (solemnly affirm) that \nI will at all times well and truly serve the Republic of Ghana in the office of \n____________________ and that I will uphold, preserve, protect and defend the Constitution \nof the Republic of Ghana as by law established. (So help me God).  \nTo be sworn before the President or such other person as the President may designate . \nTHE SPEAKER’S OATH  \nI, ___________________ do (in the name of the Almighty God swear) (solemnly affirm) that \nI will bear true faith and allegiance to the Republic of Ghana as by law established; that I will \nuphold the integrity of the Republic of Ghana; that I will faithfully and conscientiously \ndischarge my duties as Speaker of Parliament; and that I will uphol